<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c553dc01b3ee4a88d20033531ec0b9f8562442d49c98090c92a88dc6b5fd0abf
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 10:41:55
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: 18.24 K (0.13%)
Current PnL: -21.96 L (-14.39%)
CY Booked + Current PnL: -8.07 L (-5.29%)
-------------------
Total profit:  1.83 L
Total loss:  -23.78 L
-------------------
Total Booked + Current PnL: 18.91 L (15.06%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.66%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.37 L (60.74%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,-0.10,-14.20,16.66,0.09,14405.0,-14310.0,86467.0,100.64,49.0,M-SC,3.54,253.0,-0.99,0.60,13.60,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-3.81,-10.72,12.03,0.02,18749.0,-18710.0,155849.0,137.72,61.0,M-SC,9.04,234.0,-1.00,1.08,35.56,OX40N,NTT,PAINTS
53,NESTLEIND,1377.0,0.01,12.57,7.84,21.40,23426.0,33369.0,298795.0,4.42,58.0,X-LC,7.68,12.0,1.42,2.08,20.05,XY25,NTT,FMCG
83,VOLTAS,1530.0,-1.15,7.25,11.60,19.69,23855.0,13908.0,205650.0,-3.15,49.0,X-MC,3.33,78.0,0.58,1.43,14.54,XY25,NTT,AC
52,MEDANTA,1486.0,-0.32,5.15,18.00,24.08,24027.0,6536.0,133486.0,-3.40,50.0,X-SC,5.53,89.0,0.27,0.93,25.77,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,6.14,5.70,25.28,32.42,67843.0,14469.0,268365.0,-13.06,57.0,X-SC,7.45,85.0,0.21,1.87,21.66,X40,ATH,FMCG
58,RAJOOENG,143.10,4.39,-24.18,84.12,39.61,65378.0,-24780.0,77720.0,-48.73,41.0,H-SC,20.45,135.0,-0.38,0.54,9.54,AR,ATH,MISC
67,SIS,528.00,2.77,-20.73,52.98,21.26,46629.0,-23019.0,88013.0,2080.83,62.0,H-SC,3.77,166.0,-0.49,0.61,18.91,OX40N,NTT,MISC
68,SONACOMS,806.63,2.14,-10.38,55.71,39.54,50506.0,-10500.0,90659.0,-29.96,71.0,M-SC,6.65,220.0,-0.21,0.63,27.84,AR,ATH,AUTO
46,JSWINFRA,342.00,1.80,-7.48,24.54,15.23,45486.0,-14992.0,185355.0,-20.80,37.0,X-MC,6.88,66.0,-0.33,1.29,22.35,X40N,NTT,REALTY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-9.78,-13.17,109.51,81.92,109162.0,-15116.0,99682.0,-49.09,24.0,M-SC,3.36,236.0,-0.14,0.69,18.01,XR,NTT,DURABLES
39,INDIGOPNTS,1408.0,-3.81,-10.72,12.03,0.02,18749.0,-18710.0,155849.0,137.72,61.0,M-SC,9.04,234.0,-1.00,1.08,35.56,OX40N,NTT,PAINTS
28,GREENPANEL,537.0,-1.47,-32.18,111.04,43.14,137881.0,-58906.0,124172.0,178.42,33.0,M-SC,5.84,240.0,-0.43,0.86,16.80,XY24,NTT,MISC
70,SURYODAY,216.0,-1.17,-19.22,52.31,23.03,75670.0,-34415.0,144656.0,57.89,49.0,H-SC,9.47,167.0,-0.45,1.01,43.53,XR,NTT,BANKS
83,VOLTAS,1530.0,-1.15,7.25,11.60,19.69,23855.0,13908.0,205650.0,-3.15,49.0,X-MC,3.33,78.0,0.58,1.43,14.54,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,ZYDUSLIFE,1286.17,0.35,-0.47,36.28,35.63,75673.0,-994.0,208580.0,-14.53,45.0,H-MC,3.98,119.0,-0.01,1.45,16.12,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.13,-4.32,117.31,107.91,165712.0,-6384.0,141260.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.98,1.01,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-3.81,-10.72,12.03,0.02,18749.0,-18710.0,155849.0,137.72,61.0,M-SC,9.04,234.0,-1.00,1.08,35.56,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.10,-14.20,16.66,0.09,14405.0,-14310.0,86467.0,100.64,49.0,M-SC,3.54,253.0,-0.99,0.60,13.60,OX40N,NTT,DURABLES
67,SIS,528.00,2.77,-20.73,52.98,21.26,46629.0,-23019.0,88013.0,2080.83,62.0,H-SC,3.77,166.0,-0.49,0.61,18.91,OX40N,NTT,MISC
47,KANSAINER,340.00,0.33,-21.64,44.81,13.47,94688.0,-58356.0,211311.0,-67.73,35.0,H-SC,1.49,158.0,-0.62,1.47,7.45,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.03,2.65,67.83,72.27,117977.0,4483.0,173931.0,-11.98,60.0,M-LC,3.85,99.0,0.04,1.21,8.98,XR,NTT,IT
38,INDIAMART,4810.62,0.49,1.03,104.62,106.72,130360.0,1267.0,124603.0,-50.06,41.0,H-SC,3.11,139.0,0.01,0.87,24.65,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.03,2.65,67.83,72.27,117977.0,4483.0,173931.0,-11.98,60.0,M-LC,3.85,99.0,0.04,1.21,8.98,XR,NTT,IT
38,INDIAMART,4810.62,0.49,1.03,104.62,106.72,130360.0,1267.0,124603.0,-50.06,41.0,H-SC,3.11,139.0,0.01,0.87,24.65,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.35,-0.47,36.28,35.63,75673.0,-994.0,208580.0,-14.53,45.0,H-MC,3.98,119.0,-0.01,1.45,16.12,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.13,-4.32,117.31,107.91,165712.0,-6384.0,141260.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.98,1.01,OX40N,ATH,CABLES
37,IEX,219.00,-0.18,-3.96,54.76,48.63,105155.0,-7925.0,192029.0,-36.00,54.0,H-SC,15.60,136.0,-0.08,1.34,7.13,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.57,-35.11,108.35,35.20,90470.0,-45172.0,83498.0,9.46,28.0,X-SC,13.17,92.0,-0.50,0.58,0.57,X40,NTT,FOOTWEAR
76,TMPV,600.00,0.15,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,29.0,X-LC,1.46,3.0,-0.26,1.64,2.09,XY24,NTT,AUTO
55,PGHH,17907.65,0.01,-4.56,40.16,33.76,76969.0,-9165.0,191655.0,-32.64,33.0,X-MC,3.73,57.0,-0.12,1.33,0.40,X40,ATH,FMCG
27,GLAXO,3466.20,-0.41,-0.48,39.82,39.15,78971.0,-964.0,198320.0,-20.01,33.0,X-MC,10.09,60.0,-0.01,1.38,28.38,X40,ATH,PHARMA
46,JSWINFRA,342.00,1.80,-7.48,24.54,15.23,45486.0,-14992.0,185355.0,-20.80,37.0,X-MC,6.88,66.0,-0.33,1.29,22.35,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.06,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,43.0,X-LC,1.02,5.0,-0.14,1.37,3.83,X40,NTT,FMCG
76,TMPV,600.0,0.15,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,29.0,X-LC,1.46,3.0,-0.26,1.64,2.09,XY24,NTT,AUTO
3,ACC,3906.0,-0.50,-21.20,108.32,64.15,203100.0,-50451.0,187500.0,-53.69,54.0,X-SC,1.48,82.0,-0.25,1.30,5.18,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.51,3.62,26.39,30.97,48331.0,6402.0,183140.0,-20.80,58.0,X-MC,1.83,75.0,0.13,1.27,16.64,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-1.07,-11.79,42.28,25.51,69407.0,-21935.0,164160.0,0.72,62.0,H-LC,1.87,49.0,-0.32,1.14,20.97,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PGHH,17907.65,0.01,-4.56,40.16,33.76,76969.0,-9165.0,191655.0,-32.64,33.0,X-MC,3.73,57.0,-0.12,1.33,0.40,X40,ATH,FMCG
11,BATAINDIA,2096.00,0.57,-35.11,108.35,35.20,90470.0,-45172.0,83498.0,9.46,28.0,X-SC,13.17,92.0,-0.50,0.58,0.57,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-0.17,-4.89,32.54,26.07,50678.0,-8000.0,155740.0,-29.94,39.0,X-MC,9.36,55.0,-0.16,1.08,1.04,X40,ATH,APPARELS
9,BAJAJHFL,181.50,1.15,-13.13,70.89,48.45,123479.0,-26322.0,174184.0,-19.44,39.0,X-MC,7.28,64.0,-0.21,1.21,1.19,X40N,ATH,FINANCE
76,TMPV,600.00,0.15,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,29.0,X-LC,1.46,3.0,-0.26,1.64,2.09,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.30,-27.39,96.72,42.84,45655.0,-17803.0,47203.0,-52.62,42.0,X-SC,37.54,83.0,-0.39,0.33,2.71,XY24,NTT,MISC
60,RELAXO,1176.00,-0.58,-46.90,186.83,52.31,144009.0,-68080.0,77080.0,-43.47,38.0,X-SC,4.44,91.0,-0.47,0.54,3.18,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.57,-35.11,108.35,35.20,90470.0,-45172.0,83498.0,9.46,28.0,X-SC,13.17,92.0,-0.50,0.58,0.57,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.63,-5.39,28.51,21.58,37801.0,-7551.0,132588.0,-52.33,37.0,X-MC,6.56,56.0,-0.20,0.92,17.38,X40N,ATH,IT
52,MEDANTA,1486.00,-0.32,5.15,18.00,24.08,24027.0,6536.0,133486.0,-3.40,50.0,X-SC,5.53,89.0,0.27,0.93,25.77,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.19,-14.22,39.06,19.28,115914.0,-49198.0,296758.0,-25.24,62.0,X-LC,7.23,1.0,-0.42,2.06,9.71,X40,ATH,IT
41,INFY,2275.00,0.55,6.63,45.24,54.87,150940.0,20755.0,333643.0,-17.09,62.0,X-LC,2.62,2.0,0.14,2.32,15.47,X40,BTT,IT
76,TMPV,600.00,0.15,-14.74,66.76,42.18,157333.0,-40741.0,235669.0,-23.08,29.0,X-LC,1.46,3.0,-0.26,1.64,2.09,XY24,NTT,AUTO
82,VBL,671.64,0.13,-5.85,44.10,35.67,131141.0,-18470.0,297372.0,-17.02,54.0,X-LC,5.77,4.0,-0.14,2.07,7.20,X40N,ATH,FMCG
43,ITC,452.00,0.06,-1.64,12.28,10.44,24173.0,-3291.0,196847.0,-41.15,43.0,X-LC,1.02,5.0,-0.14,1.37,3.83,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.68,-13.25,109.42,81.67,89048.0,-12428.0,81382.0,7316.67,50.0,L-SC,19.27,271.0,-0.14,0.57,57.60,XR,NTT,CERAMICS
68,SONACOMS,806.63,2.14,-10.38,55.71,39.54,50506.0,-10500.0,90659.0,-29.96,71.0,M-SC,6.65,220.0,-0.21,0.63,27.84,AR,ATH,AUTO
51,MASFIN,398.61,0.05,-5.80,29.57,22.05,27292.0,-5685.0,92295.0,-18.70,48.0,H-SC,6.99,164.0,-0.21,0.64,34.16,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.17,-19.22,52.31,23.03,75670.0,-34415.0,144656.0,57.89,49.0,H-SC,9.47,167.0,-0.45,1.01,43.53,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-3.81,-10.72,12.03,0.02,18749.0,-18710.0,155849.0,137.72,61.0,M-SC,9.04,234.0,-1.00,1.08,35.56,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-3.81,-10.72,12.03,0.02,18749.0,-18710.0,155849.0,137.72,61.0,M-SC,9.04,234.0,-1.00,1.08,35.56,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,1.08,1.54,19.11,20.95,48855.0,3881.0,255649.0,0.49,77.0,X-LC,14.60,31.0,0.08,1.78,37.73,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.06,-37.16,111.47,32.89,54083.0,-28688.0,48518.0,-699.18,64.0,L-MC,5.63,259.0,-0.53,0.34,33.62,XR,NTT,BANKS
0,5PAISA,593.00,-0.17,-34.65,72.76,12.90,103895.0,-75704.0,142792.0,110.34,59.0,H-SC,13.38,161.0,-0.73,0.99,18.73,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,0.12,7.35,12.50,20.77,31418.0,17200.0,251344.0,-2.02,66.0,X-MC,3.79,62.0,0.55,1.75,14.77,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.74
2,50,75.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.64
MC    29.19
LC    25.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.85
X40      21.23
X40N     11.98
XR        9.56
AR        9.11
XY25      9.06
OX40N     7.64
X200      1.85
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.73
X-LC    20.03
M-SC    11.95
X-SC     8.35
H-MC     4.76
H-LC     2.99
L-SC     1.41
M-MC     1.36
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.85,-3.93,37.54
IT,13.76,-14.19,73.15
FINANCE,9.42,-14.39,62.72
MISC,7.44,-21.39,71.13
ELECTRICAL,5.98,-9.24,49.26
PAINTS,5.95,-7.94,23.97
INSURANCE,4.41,0.46,34.62
PHARMA,3.86,-0.55,32.74
AUTO,3.38,-18.29,69.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3032069.0,21
XR,1291092.0,13
AR,1262526.0,10
X40,984473.0,14
X40N,726413.0,9
OX40N,692166.0,10
XY25,337706.0,6
SR,279034.0,2
MH,73586.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3537681.0,25
M-SC,1339669.0,15
X-MC,1213978.0,16
X-LC,882762.0,11
X-SC,719738.0,8
H-MC,394614.0,3
L-SC,260037.0,3
H-LC,127126.0,2
M-LC,117977.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1213813.0      6
           AR         867295.0      5
           XR         788382.0      7
M-SC       XY24       773152.0      6
X-MC       X40        439579.0      7
X-LC       X40        386581.0      5
X-MC       XY24       358530.0      3
H-SC       OX40N      315571.0      4
M-SC       OX40N      289412.0      5
X-SC       X40N       288643.0      3
H-SC       SR         279034.0      2
X-SC       XY24       272782.0      3
X-MC       X40N       238184.0      4
X-LC       XY24       225693.0      2
H-MC       AR         206515.0      2
X-LC       X40N       199586.0      2
H-MC       XY24       188099.0      1
X-MC       XY25       177685.0      2
L-SC       XR         172854.0      2
X-SC       X40        158313.0      2
M-SC       XR         157796.0      2
           AR         119309.0      2
M-LC       XR         117977.0      1
L-SC       OX40N       87183.0      1
H-SC       MH          73586.0      1
X-LC       XY25        70902.0      2
H-LC       AR          69407.0      1
           X200        57719.0      1
L-MC       XR          54083.0      1
M-MC       XY25        49279.0      1
L-LC       XY25        39840.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
